In [17]:
# ═══════════════════════════════════════════════════════════════════
# DETECTOR AUTOMÁTICO DE ARQUIVOS - v1.0 CORRIGIDO
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import xlrd
import re
from pathlib import Path
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# GUI
import tkinter as tk
from tkinter import filedialog

print("╔" + "="*78 + "╗")
print("║" + " 🔍 DETECTOR AUTOMÁTICO DE ARQUIVOS v1.0".center(78) + "║")
print("╚" + "="*78 + "╝")
print("\n✅ Imports carregados")

╔==============================================================================╗
║                    🔍 DETECTOR AUTOMÁTICO DE ARQUIVOS v1.0                    ║
╚==============================================================================╝

✅ Imports carregados


In [18]:
# ═══════════════════════════════════════════════════════════════════
# FILEMANAGER STANDALONE
# ═══════════════════════════════════════════════════════════════════

class FileManagerSimples:
    def __init__(self, base_path=None):
        self.base_path = Path(base_path) if base_path else Path.cwd()

        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs'
        }

        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        print(f"✅ FileManager inicializado")
        print(f"   Base: {self.base_path}")
        print(f"   Timestamp: {self.timestamp}")

    def salvar(self, df, nome, tipo='xlsx'):
        arquivo = self.pastas['processados'] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        print(f"💾 Salvo: {arquivo.name}")
        return arquivo

fm = FileManagerSimples()

✅ FileManager inicializado
   Base: C:\Users\fpsou\PycharmProjects\AIVI-RECALCULOBatentesLimites
   Timestamp: 20251015_154525


In [19]:
# ═══════════════════════════════════════════════════════════════════
# PADRÕES REGEX
# ═══════════════════════════════════════════════════════════════════

PADROES_CAMPO = {
    'Centro': {
        'regex': r'^[5-9]\d{3}$',
        'exemplos': ['5025', '5065', '5174']
    },
    'Codigo_Produto': {
        'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$',
        'exemplos': ['10.123.456', '1234567']
    },
    'Codigo_Grupo': {
        'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$|^[A-Z_]+$',
        'exemplos': ['10.123.456', 'DIESEL_S10']
    },
    'Doc_Transporte': {
        'regex': r'^\d{10}$',
        'exemplos': ['1234567890']
    },
    'Sigla_Base': {
        'regex': r'^[A-Z]{4,10}$',
        'exemplos': ['BABET', 'BAPLAN']
    },
    'CNPJ': {
        'regex': r'^\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}$|^\d{14}$',
        'exemplos': ['12.345.678/0001-99']
    },
    'Percentual': {
        'regex': r'^-?\d+([.,]\d+)?%?$',
        'exemplos': ['10.5%', '-5.2']
    },
    'Monetario': {
        'regex': r'^R\$\s?-?\d{1,3}(\.\d{3})*(,\d{2})?$|^-?\d+([.,]\d{2})?$',
        'exemplos': ['R$ 1.234,56', '1234.56']
    },
    'Numero_Inteiro': {
        'regex': r'^-?\d+$',
        'exemplos': ['123', '-456']
    },
    'Numero_Decimal': {
        'regex': r'^-?\d+([.,]\d+)?$',
        'exemplos': ['123.45', '123,45']
    },
    'Data_ISO': {
        'regex': r'^\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])$',
        'exemplos': ['2024-01-15']
    },
    'Data_BR': {
        'regex': r'^(0[1-9]|[12]\d|3[01])/(0[1-9]|1[0-2])/\d{4}$',
        'exemplos': ['15/01/2024']
    }
}

def detectar_tipo_campo(valores_amostra):
    valores_str = [str(v).strip() for v in valores_amostra if pd.notna(v)]

    if not valores_str:
        return {'tipo': 'VAZIO', 'confianca': 0.0}

    resultados = []
    for nome_tipo, info in PADROES_CAMPO.items():
        matches = sum(1 for v in valores_str if re.match(info['regex'], v))
        prop = matches / len(valores_str)
        resultados.append({'tipo': nome_tipo, 'confianca': prop, 'matches': matches, 'total': len(valores_str)})

    melhor = max(resultados, key=lambda x: x['confianca'])
    if melhor['confianca'] < 0.70:
        melhor['tipo'] = 'TEXTO_GENERICO'

    return melhor

print(f"✅ {len(PADROES_CAMPO)} padrões carregados")

✅ 12 padrões carregados


In [20]:
# ═══════════════════════════════════════════════════════════════════
# GUI PARA SELEÇÃO DE ARQUIVO
# ═══════════════════════════════════════════════════════════════════

def selecionar_arquivo_gui():
    """Abre janela GUI para selecionar arquivo"""
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)

    arquivo = filedialog.askopenfilename(
        title="Selecione o arquivo Excel",
        initialdir=fm.pastas['entrada'],
        filetypes=[
            ("Arquivos Excel", "*.xlsx *.xls *.xlsm"),
            ("Todos os arquivos", "*.*")
        ]
    )

    root.destroy()

    if not arquivo:
        raise ValueError("❌ Nenhum arquivo selecionado")

    return Path(arquivo)

# EXECUTAR SELEÇÃO
print("📂 Abrindo janela de seleção...")
print("   (A janela pode aparecer atrás do navegador)")

arquivo_selecionado = selecionar_arquivo_gui()

print(f"\n✅ Arquivo selecionado:")
print(f"   Nome: {arquivo_selecionado.name}")
print(f"   Tamanho: {arquivo_selecionado.stat().st_size / 1024:.1f} KB")
print(f"   Caminho: {arquivo_selecionado}")

📂 Abrindo janela de seleção...
   (A janela pode aparecer atrás do navegador)

✅ Arquivo selecionado:
   Nome: ysmm_centros_br.xlsx
   Tamanho: 114.7 KB
   Caminho: F:\Downloads\AIVI\ysmm_centros_br.xlsx


In [21]:
# ═══════════════════════════════════════════════════════════════════
# CARREGAR WORKBOOK
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 1: CARREGAMENTO")
print("="*80)

try:
    workbook = xlrd.open_workbook(str(arquivo_selecionado))
    sheets = workbook.sheet_names()
    metodo_carga = 'xlrd'
    print(f"✅ Carregado com xlrd (XLS)")
except:
    workbook = pd.ExcelFile(str(arquivo_selecionado))
    sheets = workbook.sheet_names
    metodo_carga = 'pandas'
    print(f"✅ Carregado com pandas (XLSX/XLSM)")

print(f"\n📊 Sheets: {len(sheets)}")
for i, sheet in enumerate(sheets, 1):
    print(f"   {i}. {sheet}")


FASE 1: CARREGAMENTO
✅ Carregado com pandas (XLSX/XLSM)

📊 Sheets: 3
   1. Sheet1
   2. Sheet2
   3. Sheet3


In [22]:
# ═══════════════════════════════════════════════════════════════════
# DETECTAR SHEET COM DADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 2: DETECÇÃO DE SHEET")
print("="*80)

def avaliar_sheet(sheet_name):
    try:
        if metodo_carga == 'xlrd':
            sheet = workbook.sheet_by_name(sheet_name)
            linhas = sheet.nrows
            colunas = sheet.ncols
        else:
            df_sample = pd.read_excel(workbook, sheet_name=sheet_name, nrows=50)
            linhas = len(df_sample)
            colunas = len(df_sample.columns)

        score = 0.0
        if linhas > 10: score += 1.0
        if colunas > 5: score += 1.0
        if linhas > 100: score += 0.5

        return {'sheet': sheet_name, 'score': score, 'linhas': linhas, 'colunas': colunas}
    except:
        return {'sheet': sheet_name, 'score': 0.0, 'linhas': 0, 'colunas': 0}

avaliacoes = [avaliar_sheet(s) for s in sheets]
avaliacoes = [a for a in avaliacoes if a['score'] > 0]

if not avaliacoes:
    raise ValueError("❌ Nenhuma sheet com dados")

melhor_sheet = max(avaliacoes, key=lambda x: x['score'])

print(f"🎯 Sheet selecionada: '{melhor_sheet['sheet']}'")
print(f"   Linhas: {melhor_sheet['linhas']:,} | Colunas: {melhor_sheet['colunas']} | Score: {melhor_sheet['score']:.2f}")

sheet_nome = melhor_sheet['sheet']


FASE 2: DETECÇÃO DE SHEET
🎯 Sheet selecionada: 'Sheet1'
   Linhas: 50 | Colunas: 30 | Score: 2.00


In [23]:
# ═══════════════════════════════════════════════════════════════════
# DETECTAR CABEÇALHO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 3: DETECÇÃO DE CABEÇALHO")
print("="*80)

if metodo_carga == 'xlrd':
    sheet = workbook.sheet_by_name(sheet_nome)
    linhas_amostra = [sheet.row_values(i) for i in range(min(100, sheet.nrows))]
else:
    df_amostra = pd.read_excel(workbook, sheet_name=sheet_nome, nrows=100, header=None)
    linhas_amostra = df_amostra.values.tolist()

def avaliar_linha_cabecalho(linha, idx):
    celulas = [str(c).strip() for c in linha if str(c).strip()]
    if not celulas:
        return 0.0

    score = 0.0
    score += (len(celulas) / len(linha)) * 2.0

    tem_texto = sum(1 for c in celulas if re.search(r'[a-zA-Z]', c))
    score += (tem_texto / len(celulas)) * 1.5

    keywords = ['centro', 'produto', 'código', 'data', 'quantidade', 'valor', 'sigla']
    texto = ' '.join(celulas).lower()
    kw_encontradas = sum(1 for kw in keywords if kw in texto)
    score += (kw_encontradas / len(keywords)) * 3.0

    tamanho_medio = np.mean([len(c) for c in celulas])
    if 5 <= tamanho_medio <= 50:
        score += 1.0

    contador = Counter(celulas)
    if all(n == 1 for n in contador.values()):
        score += 1.5

    if idx < 50:
        score += (50 - idx) / 100

    return score

scores = [{'linha_idx': idx, 'score': avaliar_linha_cabecalho(linha, idx), 'conteudo': linha[:3]}
          for idx, linha in enumerate(linhas_amostra)]
scores = sorted(scores, key=lambda x: x['score'], reverse=True)

print("🏆 Top 3 candidatos:")
for i, item in enumerate(scores[:3], 1):
    print(f"   {i}. Linha {item['linha_idx']+1} - Score: {item['score']:.2f}")

linha_cabecalho = scores[0]['linha_idx']
linha_dados_inicio = linha_cabecalho + 1

print(f"\n🎯 Cabeçalho: Linha {linha_cabecalho + 1} (Excel)")
print(f"   Dados: Linha {linha_dados_inicio + 1} (Excel)")


FASE 3: DETECÇÃO DE CABEÇALHO
🏆 Top 3 candidatos:
   1. Linha 1 - Score: 7.79
   2. Linha 2 - Score: 5.17
   3. Linha 5 - Score: 5.09

🎯 Cabeçalho: Linha 1 (Excel)
   Dados: Linha 2 (Excel)


In [24]:
# ═══════════════════════════════════════════════════════════════════
# EXTRAIR DADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 4: EXTRAÇÃO")
print("="*80)

if metodo_carga == 'xlrd':
    sheet = workbook.sheet_by_name(sheet_nome)
    data = [sheet.row_values(i) for i in range(sheet.nrows)]
    df_bruto = pd.DataFrame(data)

    cabecalho_bruto = df_bruto.iloc[linha_cabecalho].tolist()
    df_bruto = df_bruto.iloc[linha_dados_inicio:].copy()
    df_bruto.columns = cabecalho_bruto
else:
    df_bruto = pd.read_excel(workbook, sheet_name=sheet_nome, header=linha_cabecalho)

df_bruto = df_bruto.reset_index(drop=True)

print(f"✅ Extraído: {len(df_bruto):,} registros × {len(df_bruto.columns)} colunas")


FASE 4: EXTRAÇÃO
✅ Extraído: 555 registros × 30 colunas


In [25]:
# ═══════════════════════════════════════════════════════════════════
# LIMPEZA DE ESTRUTURA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 5: LIMPEZA")
print("="*80)

df = df_bruto.copy()

# Colunas vazias
colunas_vazias = df.columns[df.isna().all()].tolist()
if colunas_vazias:
    print(f"🗑️  {len(colunas_vazias)} colunas vazias removidas")
    df = df.drop(columns=colunas_vazias)

# Linhas vazias
linhas_vazias = df.isna().all(axis=1).sum()
if linhas_vazias > 0:
    print(f"🗑️  {linhas_vazias} linhas vazias removidas")
    df = df.dropna(how='all')

# Limpar nomes
colunas_limpas = []
for col in df.columns:
    col_limpo = str(col).strip().lstrip("'").replace('\n', ' ')
    col_limpo = ' '.join(col_limpo.split())
    colunas_limpas.append(col_limpo)
df.columns = colunas_limpas

# Duplicadas
contagem = Counter(colunas_limpas)
duplicadas = {c: n for c, n in contagem.items() if n > 1}

if duplicadas:
    print(f"⚠️  {len(duplicadas)} colunas duplicadas renomeadas")
    colunas_finais = []
    contador = {}
    for col in colunas_limpas:
        if col in duplicadas:
            if col not in contador:
                contador[col] = 0
                colunas_finais.append(col)
            else:
                contador[col] += 1
                colunas_finais.append(f"{col}_dup{contador[col]}")
        else:
            colunas_finais.append(col)
    df.columns = colunas_finais

# Remover totais
padroes_remover = [r'(?i)^total', r'(?i)^resultado', r'(?i)^soma']
linhas_remover = [idx for idx, row in df.iterrows()
                  if any(re.search(p, str(row.iloc[0]).strip().lower()) for p in padroes_remover)]

if linhas_remover:
    print(f"🗑️  {len(linhas_remover)} linhas de totais removidas")
    df = df.drop(index=linhas_remover)

df = df.reset_index(drop=True)
df_limpo = df.copy()

print(f"\n✅ Limpeza OK: {len(df_limpo):,} registros × {len(df_limpo.columns)} colunas")


FASE 5: LIMPEZA

✅ Limpeza OK: 555 registros × 30 colunas


In [26]:
# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO DE TIPOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 6: DETECÇÃO DE TIPOS")
print("="*80)

tipos_detectados = {}

for col in df_limpo.columns:
    valores_amostra = df_limpo[col].dropna().head(100).tolist()
    tipo_info = detectar_tipo_campo(valores_amostra)
    tipos_detectados[col] = tipo_info

    emoji = "✅" if tipo_info['confianca'] >= 0.90 else "⚠️ " if tipo_info['confianca'] >= 0.70 else "❓"
    print(f"{emoji} {col[:40]:40s} → {tipo_info['tipo']:20s} ({tipo_info['confianca']:.0%})")

print(f"\n✅ {len(tipos_detectados)} colunas analisadas")


FASE 6: DETECÇÃO DE TIPOS
✅ Centro                                   → Percentual           (100%)
❓ Sigla                                    → TEXTO_GENERICO       (40%)
✅ Regional/Região                          → Codigo_Grupo         (100%)
❓ Nome 1                                   → TEXTO_GENERICO       (0%)
❓ Rua                                      → TEXTO_GENERICO       (0%)
❓ Número                                   → TEXTO_GENERICO       (65%)
❓ Compl                                    → TEXTO_GENERICO       (3%)
❓ Bairro                                   → TEXTO_GENERICO       (31%)
❓ Local.Resd.Difer.                        → TEXTO_GENERICO       (2%)
❓ Código Postal                            → TEXTO_GENERICO       (0%)
❓ Cidade                                   → TEXTO_GENERICO       (58%)
✅ Estado                                   → Codigo_Grupo         (100%)
✅ Filial                                   → Percentual           (100%)
✅ CNPJ                                

In [27]:
# ═══════════════════════════════════════════════════════════════════
# EXPLORAÇÃO ESTATÍSTICA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 7: EXPLORAÇÃO")
print("="*80)

print(f"\n📊 Resumo:")
print(f"   Registros: {len(df_limpo):,}")
print(f"   Colunas: {len(df_limpo.columns)}")
print(f"   Memória: {df_limpo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Duplicatas: {df_limpo.duplicated().sum():,}")

print(f"\n🔍 Valores Nulos:")
nulos = df_limpo.isna().sum()
colunas_com_nulos = nulos[nulos > 0].sort_values(ascending=False)

if len(colunas_com_nulos) == 0:
    print("   ✅ Nenhuma coluna com nulos")
else:
    print(f"   ⚠️  {len(colunas_com_nulos)} colunas com nulos:")
    for col, qtd in colunas_com_nulos.head(5).items():
        print(f"      • {col}: {qtd:,} ({qtd/len(df_limpo)*100:.1f}%)")

print(f"\n🔬 Tipos Detectados:")
tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['tipo']
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

for tipo, count in sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True):
    print(f"   • {tipo}: {count} colunas")


FASE 7: EXPLORAÇÃO

📊 Resumo:
   Registros: 555
   Colunas: 30
   Memória: 0.73 MB
   Duplicatas: 0

🔍 Valores Nulos:
   ⚠️  23 colunas com nulos:
      • Local.Resd.Difer.: 507 (91.4%)
      • Compl: 484 (87.2%)
      • Regional/Região: 463 (83.4%)
      • Longitude: 379 (68.3%)
      • Latitude: 379 (68.3%)

🔬 Tipos Detectados:
   • TEXTO_GENERICO: 13 colunas
   • Percentual: 11 colunas
   • Codigo_Grupo: 6 colunas


In [28]:
# ═══════════════════════════════════════════════════════════════════
# EXPORTAÇÃO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("FASE 8: EXPORTAÇÃO")
print("="*80)

nome_base = arquivo_selecionado.stem

# 1. Dados limpos
arquivo_limpo = fm.salvar(df_limpo, f"{nome_base}_Limpo", tipo='xlsx')

# 2. Dicionário
registros_dict = []
for col in df_limpo.columns:
    tipo_info = tipos_detectados.get(col, {})
    registros_dict.append({
        'Coluna': col,
        'Tipo_Detectado': tipo_info.get('tipo', 'DESCONHECIDO'),
        'Confianca': tipo_info.get('confianca', 0.0),
        'Dtype': str(df_limpo[col].dtype),
        'Unicos': df_limpo[col].nunique(),
        'Nulos': df_limpo[col].isna().sum(),
        'Nulos_%': (df_limpo[col].isna().sum() / len(df_limpo)) * 100
    })

df_dict = pd.DataFrame(registros_dict)
arquivo_dict = fm.pastas['outputs'] / f"DICT_{nome_base}_{fm.timestamp}.xlsx"
df_dict.to_excel(arquivo_dict, index=False)
print(f"💾 Dicionário: {arquivo_dict.name}")

# 3. Log
log = {
    'Arquivo': arquivo_selecionado.name,
    'Sheet': sheet_nome,
    'Linha_Cabecalho': linha_cabecalho + 1,
    'Registros_Original': len(df_bruto),
    'Registros_Final': len(df_limpo),
    'Colunas': len(df_limpo.columns),
    'Timestamp': fm.timestamp
}
df_log = pd.DataFrame([log])
arquivo_log = fm.pastas['logs'] / f"LOG_{nome_base}_{fm.timestamp}.xlsx"
df_log.to_excel(arquivo_log, index=False)
print(f"💾 Log: {arquivo_log.name}")

print("\n✅ EXPORTAÇÃO CONCLUÍDA")

# Variável global
df_resultado = df_limpo.copy()


FASE 8: EXPORTAÇÃO
💾 Salvo: ysmm_centros_br_Limpo_20251015_154525.xlsx
💾 Dicionário: DICT_ysmm_centros_br_20251015_154525.xlsx
💾 Log: LOG_ysmm_centros_br_20251015_154525.xlsx

✅ EXPORTAÇÃO CONCLUÍDA


In [29]:
# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO FINAL
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📋 RELATÓRIO FINAL")
print("="*80)

print(f"\n📁 Arquivo: {arquivo_selecionado.name}")
print(f"📊 Sheet: {sheet_nome}")
print(f"📍 Cabeçalho: Linha {linha_cabecalho + 1}")
print(f"📍 Dados: Linha {linha_dados_inicio + 1}")

print(f"\n📊 Resultado:")
print(f"   {len(df_resultado):,} registros")
print(f"   {len(df_resultado.columns)} colunas")

print(f"\n💾 Arquivos gerados:")
print(f"   1. {arquivo_limpo.name}")
print(f"   2. {arquivo_dict.name}")
print(f"   3. {arquivo_log.name}")

print("\n✅ PROCESSAMENTO CONCLUÍDO!")
print("="*80)
print(f"\n💡 Use 'df_resultado' para continuar análises")


📋 RELATÓRIO FINAL

📁 Arquivo: ysmm_centros_br.xlsx
📊 Sheet: Sheet1
📍 Cabeçalho: Linha 1
📍 Dados: Linha 2

📊 Resultado:
   555 registros
   30 colunas

💾 Arquivos gerados:
   1. ysmm_centros_br_Limpo_20251015_154525.xlsx
   2. DICT_ysmm_centros_br_20251015_154525.xlsx
   3. LOG_ysmm_centros_br_20251015_154525.xlsx

✅ PROCESSAMENTO CONCLUÍDO!

💡 Use 'df_resultado' para continuar análises


In [30]:
df_resultado

,Centro,Sigla,Regional/Região,Nome 1,Rua,Número,Compl,Bairro,Local.Resd.Difer.,Código Postal,...,Centro Inativo,Centro Fornecedor,Longitude,Latitude,Centro de Custo,Centro de Lucro,Cliente Centro,Fornecedor Centro,Endereço,Agrpmto.estrut.reg.
0,1,Centro,NaN,Centro 0001,RUA CORREIA VASQUES,250,NaN,CIDADE NOVA,NaN,20211-140,...,N,NÃO,NaN,NaN,NaN,NaN,NaN,NaN,1409832,NaN
1,2,Centro,NaN,Centro 0002 - padrão PS,NaN,NaN,NaN,NaN,NaN,NaN,...,N,NÃO,NaN,NaN,NaN,NaN,NaN,NaN,23387,NaN
2,5001,Administração,NaN,Administração Central VIBRA,RUA CORREIA VASQUES,250,NaN,CIDADE NOVA,NaN,20211-140,...,N,SIM,NaN,NaN,DFA0000000,DFA0000000,NaN,5001.0,26711,GSC
3,5002,exDISGUA,NaN,exDISGUA Antiga Regional RJ,Pça Vinte e Dois de Abril,36,NaN,CENTRO,NaN,20021-370,...,S,NÃO,NaN,NaN,DFA0000000,DFA0000000,NaN,NaN,26019,NaN
4,5003,UNISP,NaN,UNISP - UNIDADE ADM. SP,R Funchal,375,NaN,Vila Olimpia,Andar 14 Conj 141 e 142,04551-060,...,N,SIM,NaN,NaN,DFA0000000,DFA0000000,905003.0,5003.0,26823,GSC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,8262,DECAR,NaN,DECAR DEPO DE LUB DE ARAUCÁRIA,"Rodovia BR-476, Km 15",440,NaN,JARDIM ALVORADA,NaN,83707-440,...,N,SIM,NaN,NaN,DOB0SAR000,DOB0SAR000,NaN,NaN,1365070,GOP
551,8323,DETER,NaN,DETER DEPOS DE LUB DE TERESINA,Rua Dep. Paulo Ferraz,S/N,NaN,DIRCEU ARCOVERDE,NaN,64077-450,...,N,SIM,NaN,NaN,DOB0NFO004,DOB0NFO004,NaN,NaN,1365073,GOP
552,8329,DEAPE,NaN,DEAPE DEPOS DE LUB DE SUAPE,"Rodovia PE-060, Km 10",S/N,NaN,INDUSTRIAL PORTO DE SUAPE,NaN,55590-000,...,N,SIM,NaN,NaN,DOB0NSU002,DOB0NSU002,NaN,NaN,1365076,GOP
553,8330,DEÇAI,NaN,DEÇAI DEPOS LUB DE AÇAILÂNDIA,"Rodovia BR-222, Km 316",S/N,NaN,POVOADO PIQUIÁ,NaN,65930-000,...,N,SIM,NaN,NaN,DOB0NSL001,DOB0NSL001,NaN,NaN,1365077,GOP
